#Tokenizers
Copyright 2021-2023 Denis Rothman, MIT License

Reference 1 for word embedding:
https://www.geeksforgeeks.org/python-word-embedding-using-word2vec/

Reference 2 for cosine similarity:
[SciKit Learn cosine similarity documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)



#Pre-Requisistes

In [19]:
!pip install gensim
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
import math
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action = 'ignore')

#Word2Vec Tokenization


In [21]:
#Development access to delete when going into production
from google.colab import drive
drive.mount('/content/drive')

with open('drive/MyDrive/files/github.txt', 'r') as f:
    github_token = f.read().strip()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
#remove token when repository goes public
!curl -H 'Authorization: token {github_token}' -L https://raw.githubusercontent.com/Denis2054/Transformers-for-NLP-and-Computer-Vision-3rd-Edition/master/Chapter10/text.txt --output "text.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  118k  100  118k    0     0   363k      0 --:--:-- --:--:-- --:--:--  363k


In [23]:
#‘text.txt’ file
sample = open("text.txt", "r")
s = sample.read()

# processing escape characters
f = s.replace("\n", " ")

data = []
# sentence parsing
for i in sent_tokenize(f):
	temp = []
	# tokenize the sentence into words
	for j in word_tokenize(i):
		temp.append(j.lower())
	data.append(temp)

# Creating Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 512,window = 5, sg = 1)
print(model2)

Word2Vec<vocab=3291, vector_size=512, alpha=0.025>


#Cosine Similarity

In [24]:
def similarity(word1,word2):
        cosine=False #default value
        try:
                a=model2[word1]
                cosine=True
        except KeyError:     #The KeyError exception is raised
                print(word1, ":[unk] key not found in dictionary")#False implied

        try:
                b=model2[word2]#a=True implied
        except KeyError:       #The KeyError exception is raised
                cosine=False   #both a and b must be true
                print(word2, ":[unk] key not found in dictionary")

        if(cosine==True):
                b=model2[word2]
                # compute cosine similarity
                dot = np.dot(a, b)
                norma = np.linalg.norm(a)
                normb = np.linalg.norm(b)
                cos = dot / (norma * normb)

                aa = a.reshape(1,512)
                ba = b.reshape(1,512)
                #print("Word1",aa)
                #print("Word2",ba)
                cos_lib = cosine_similarity(aa, ba)
                #print(cos_lib,"word similarity")

        if(cosine==False):cos_lib=0;
        return cos_lib

#Case 0: Words in the dataset and the dictionary

In [25]:
def similarity(word1, word2):
    cosine = False  # default value
    try:
        a = model2.wv[word1]
        cosine = True
    except KeyError:     # The KeyError exception is raised
        print("The word ",word1," does not exist in the dictionary")
    try:
        b = model2.wv[word2]
    except KeyError:     # The KeyError exception is raised
        print("The word ",word2," does not exist in the dictionary")
        cosine = False  # reset to False if the second word doesn't exist
    if cosine: # if both words are in the vocabulary
        return cosine_similarity([a],[b]) # sklearn cosine_similarity requires 2D arrays
    else:
        return 0 # if either word is not in the vocabulary return similarity as 0

In [26]:
word1 = "freedom"
word2 = "liberty"
print("Similarity between", word1, "and", word2, "is", similarity(word1, word2))

Similarity between freedom and liberty is [[0.99956566]]


#Case 1: Words not in the dataset or the dictionary

In [27]:
word1="corporations";word2="rights"
print("Similarity",similarity(word1,word2),word1,word2)

The word  corporations  does not exist in the dictionary
Similarity 0 corporations rights


#Case 2: Noisy Relationship

In [28]:
word1="etext";word2="declaration"
print("Similarity",similarity(word1,word2),word1,word2)

Similarity [[0.9647783]] etext declaration


#Case 3: Words in the text but not in the dictionary

In [29]:
word1="pie";word2="logic"
print("Similarity",similarity(word1,word2),word1,word2)

The word  pie  does not exist in the dictionary
The word  logic  does not exist in the dictionary
Similarity 0 pie logic


#Case 4: Rare words

In [33]:
word1="eleutheromania";word2="liberty"
print("Similarity",similarity(word1,word2),word1,word2)

The word  eleutheromania  does not exist in the dictionary
Similarity 0 eleutheromania liberty


#Case 5: Replacing rare words

In [34]:
word1="freedom";word2="liberty"
print("Similarity",similarity(word1,word2),word1,word2)

Similarity [[0.99956566]] freedom liberty
